In [50]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
amzn_ratings = pd.read_csv('Data/ratings_Beauty.csv')

In [31]:
amzn_ratings.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [32]:
amzn_ratings = amzn_ratings.sample(25000)

In [33]:
amzn_ratings.shape

(25000, 4)

In [34]:
unique_users = amzn_ratings['UserId'].unique()
print(len(unique_users))

24425


In [35]:
unique_products = amzn_ratings['ProductId'].unique()
print(len(unique_products))

17446


In [36]:
new_df = pd.DataFrame(unique_users)
new_df.columns=['UserId']

In [37]:
for product in unique_products:
    new_df[product] = None

/var/folders/f4/m2y65p_s4w1c0qg_hsv9r9dr0000gn/T/ipykernel_84097/732139935.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[product] = None


In [38]:
for j, user in enumerate(new_df['UserId'].values):
    for i in range(amzn_ratings[amzn_ratings['UserId'] == user].shape[0]):
        product_id = amzn_ratings[amzn_ratings['UserId'] == user]['ProductId'].iloc[i]
        rating = amzn_ratings[amzn_ratings['UserId'] == user]['Rating'].iloc[i]
        new_df[product_id][j] = rating
        

In [39]:
new_df.sample(5)

,UserId,B00CO9MFAS,B002IVKXWE,B008HBMQTM,B00A97YG9U,B0013CN8LW,B00461IZY2,B0041H1CXM,B003YW5HF4,B004HFBDE6,...,B001RVINAK,B000SASZ9K,B004GF8RYG,B005KF5WYO,B002S42J6E,B00AU9DFIK,B002U33KQQ,B001FSK740,B006XF9F16,B00AYAZ8FS
7189,A1ILLQ86TQO36A,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
11444,AL8XXX5OWK1CD,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
15064,A35HE2NIKKQZ4I,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2251,A4FIOQIO6D2TP,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
23872,A1739RDYJNA6HT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [40]:
user = pd.DataFrame(new_df.iloc[2142]) #randomly select user
user = user.drop(['UserId'])
user[user.notnull().values]

,2142
B0059ISG3Q,2.0


In [44]:
subset = new_df[new_df[user[user.notnull().values].index[0]].notnull()]
subset.head()

,UserId,B00CO9MFAS,B002IVKXWE,B008HBMQTM,B00A97YG9U,B0013CN8LW,B00461IZY2,B0041H1CXM,B003YW5HF4,B004HFBDE6,...,B001RVINAK,B000SASZ9K,B004GF8RYG,B005KF5WYO,B002S42J6E,B00AU9DFIK,B002U33KQQ,B001FSK740,B006XF9F16,B00AYAZ8FS
2142,A2UD0HIQC2WIM7,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [46]:
subset = subset.replace([None], 0)
subset = subset.set_index('UserId')

In [47]:
subset.head()

,B00CO9MFAS,B002IVKXWE,B008HBMQTM,B00A97YG9U,B0013CN8LW,B00461IZY2,B0041H1CXM,B003YW5HF4,B004HFBDE6,B001ECQ4R6,...,B001RVINAK,B000SASZ9K,B004GF8RYG,B005KF5WYO,B002S42J6E,B00AU9DFIK,B002U33KQQ,B001FSK740,B006XF9F16,B00AYAZ8FS
UserId,,,,,,,,,,,,,,,,,,,,,
A2UD0HIQC2WIM7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
subset['cos_similarity'] = 0.0
for user in subset.index:
    cos = cosine_similarity(subset.loc['A2UD0HIQC2WIM7'].values.reshape(1, -1), subset.loc[user].values.reshape(1, -1))

In [52]:
subset['cos_similarity']

UserId
A2UD0HIQC2WIM7    0.0
Name: cos_similarity, dtype: float64